# Level Set Method

This program takes different approaches towards solving the level set equation for interphase tracking

In [ ]:
#Setup FeniCs

try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [5]:
from dolfin import *
from math import tanh
import matplotlib.pyplot as plt
# Create mesh
nx, ny = 150, 150
mesh = UnitSquareMesh(nx, ny) #Define mesh

# Define function space
V = FunctionSpace(mesh, 'CG', 1)

# Define parameters
R = 0.1  # Radius of the circle
delta = 0.05  # Width of the interface

# Define initial condition class, this means, create a signed function, so that phi > 0 in one subset of the domain, whereas phi < 0 on the rest.
# Note that phi should be equal to zero on the interphase between the two fluids (taking the width established)

class InitialConditions(UserExpression):
    def __init__(self, R, delta, **kwargs):
        self.R = R
        self.delta = delta
        super().__init__(**kwargs)

    def eval(self, values, x):
        d = sqrt((x[0] - 0.5)**2 + (x[1] - 0.5)**2) - self.R
        values[0] = tanh(d / self.delta)

    def value_shape(self):
        return ()

# Create an initial condition function
initial_condition = InitialConditions(R, delta, degree=1)
phi_0 = interpolate(initial_condition, V)


In [ ]:
u = Expression(('0.0', '-1.0'), degree=1) #Define the velocity that will act upon the domain

# Define time parameters
T = 1.0  # Total simulation time
dt = 0.0001  # Time step size
t = 0.0  # Initial time

# Define the advection equation
phi = TrialFunction(V)
phi_n = interpolate(initial_condition, V)  # Initial condition
v = TestFunction(V)

i = 0
j = 0
# Time-stepping loop
while t < T:

    if (i%2 == 0): #Save every two timesteps

      name = "img/output" + j*"a" + ".jpg" #Save the values of phi
      a = plot(phi_n, dt, title="Level Set Equation")
      plt.colorbar(a)
      plt.savefig(name)
      plt.clf()
      j = j +1

    # Variational form for the Level Set equation
    phi = TrialFunction(V)
    a = dot(phi, v)*dx
    L = dot(phi_n , v)*dx + dt*dot(phi_n , dot(grad(v) , u))*dx


    # Create a function to store the solution
    phi = Function(V)
    def boundary(x, on_boundary):
        return (near(x[1], 0.0) or near(x[1], 1.0) or near(x[0],0) or near(x[0],1)) and on_boundary

    bcs = [DirichletBC(V, Constant(1.0), boundary)] #Define a Dirichlet BC for the boundary of the mesh
    A, b = assemble_system(a, L, bcs)

    solve(A, phi.vector(), b)

    t += dt
    phi_n.assign(phi)  # Update the solution for the next time step

    i +=1



====================Level set solved with a Crank Nicholson discretization Scheme==================


In [ ]:
"""
The only thing that changes between this implementation and the previous one,
is the discretization scheme (finite differences) used to solve the PDE.

In this case, a Crank Nicholson scheme is defined
"""
from dolfin import *
from math import tanh
# Create mesh
nx, ny = 250, 250
mesh = UnitSquareMesh(nx, ny)

# Define function space
V = FunctionSpace(mesh, 'CG', 1)

# Define parameters
R = 0.1  # Radius of the circle
delta = 0.05  # Width of the interface

# Define initial condition class
class InitialConditions(UserExpression):
    def __init__(self, R, delta, **kwargs):
        self.R = R
        self.delta = delta
        super().__init__(**kwargs)

    def eval(self, values, x):
        d = sqrt((x[0] - 0.5)**2 + (x[1] - 0.5)**2) - self.R
        values[0] = tanh(d / self.delta)

    def value_shape(self):
        return ()

# Create an initial condition function
initial_condition = InitialConditions(R, delta, degree=1)
phi_0 = interpolate(initial_condition, V)

u = Expression(('0.0', '-1.0'), degree=1)

# Define time parameters
T = 1.0  # Total simulation time
dt = 0.00001  # Time step size
t = 0.0  # Initial time

# Define the advection equation
phi = TrialFunction(V)
phi_n = interpolate(initial_condition, V)  # Initial condition
v = TestFunction(V)

i = 0
j = 0
# Time-stepping loop
while t < T:

    if (i%2 == 0):

      name = "img2/output" + str(i) + ".jpg"
      a = plot(phi_n, dt, title="Level Set Equation")
      plt.colorbar(a)
      plt.savefig(name)
      plt.clf()
      j = j +1

    phi = TrialFunction(V)
    a = dot(phi, v)*dx - dt/2 *dot(phi, dot(grad(v) , u))*dx
    L = dot(phi_n , v)*dx + dt/2 *dot(phi_n , dot(grad(v) , u))*dx


    # Create a function to store the solution
    phi = Function(V)
    def boundary(x, on_boundary):
        return (near(x[1], 0.0) or near(x[1], 1.0) or near(x[0],0) or near(x[0],1)) and on_boundary

    bcs = [DirichletBC(V, Constant(1.0), boundary)]

    A, b = assemble_system(a, L, bcs)

    solve(A, phi.vector(), b)

    t += dt
    phi_n.assign(phi)  # Update the solution for the next time step

    i +=1

===============Level Set Method with SUPG Residual Weighting =========================

In [ ]:
"""
The only thing that changes between this implementation and the previous one,
is SUPG Residual Weighting was added to the Variational formulation.
"""

from dolfin import *
from math import tanh
from math import sin
from math import pi
# Create mesh
nx, ny = 250, 250
mesh = RectangleMesh(Point(-1,-1), Point(1,1), nx,ny)

mesh = UnitSquareMesh(250,250)
# Define function space
V = FunctionSpace(mesh, 'CG', 1)

# Define parameters
R = 0.15  # Radius of the circle
delta = 0.05  # Width of the interface

# Define initial condition class
class InitialConditions(UserExpression):
    def __init__(self, R, delta, **kwargs):
        self.R = R
        self.delta = delta
        super().__init__(**kwargs)

    def eval(self, values, x):
        d = sqrt((x[0] - 0.3)**2 + (x[1] - 0.3)**2) - self.R
        values[0] = tanh(d / self.delta)

    def value_shape(self):
        return ()

# Create an initial condition function
initial_condition = InitialConditions(R, delta, degree=1)
phi_0 = interpolate(initial_condition, V)

u = Expression(("pow(sin(pi*x[0]),2) * sin(2*pi*x[1])", "-1*pow(sin(pi*x[1]),2) * sin(2*pi*x[0])"), degree=2) #Note the Velocity applied can also be a vector field

# Define time parameters
T = 1.0  # Total simulation time
dt = 0.001  # Time step size
t = 0.0  # Initial time

# Define the advection equation
phi = TrialFunction(V)
phi_n = interpolate(initial_condition, V)  # Initial condition
v = TestFunction(V)

i = 0
j = 0
# Time-stepping loop

num_cells = mesh.num_cells()
area_mesh = 1 # En el futuro tocará cambiarlo
h = area_mesh/num_cells

V2 = VectorFunctionSpace(mesh, 'P', 1)
u_function = interpolate(u, V2)
norm_u = sqrt(assemble(inner(u_function, u_function) * dx))

while t < T:

    if (i%2 == 0):

      name = "img3/output" + j*"a" + ".jpg"
      a = plot(phi_n, dt, title="Level Set Equation")
      plt.colorbar(a)
      plt.savefig(name)
      plt.clf()
      j = j +1

    phi = TrialFunction(V)

    s = h/(2*norm_u) * dot(grad(v), u)

    a = dot(phi, v)*dx - dt/2 *dot(phi, dot(grad(v) , u))*dx  + dot(phi, s)*dx + dt/2 *dot(s, dot(grad(phi), u))*dx
    L = dot(phi_n , v)*dx + dt/2 *dot(phi_n , dot(grad(v) , u))*dx + dot(phi_n, s)*dx - dt/2 *dot(s, dot(grad(phi_n), u))*dx


    # Create a function to store the solution
    phi = Function(V)
    def boundary(x, on_boundary):
        return on_boundary

    bcs = [DirichletBC(V, Constant(1.0), boundary)]

    A, b = assemble_system(a, L, bcs)

    solve(A, phi.vector(), b)
    phi_values = phi.vector().get_local()
    phi_values[phi_values > 1.0] = 1.0 #Adjust values greater than 1, to 1
    phi.vector().set_local(phi_values)
    phi.vector().apply("insert")

    t += dt
    phi_n.assign(phi)  # Update the solution for the next time step

    i +=1

